#Hyperallignment of Data
##Written by Gergana Slaveykova s1070004
##Radboud University- B3 Thesis project

In [ ]:
# Let's define functions and load the data.
# Note: procrusted() and HyperAlign() function was adapted from python package HyperTools.
# Source: https://github.com/ContextLab/hypertools

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from sklearn.svm import SVC
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix
from sklearn.manifold import TSNE

In [ ]:
#Mount Google Drive to acess data and storing purposes
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def procrustes(source, target):
    def fit(source, target):
        datas = (source, target)

        ##-------------- STEP 1: Normalize ---------------##
        ssqs = [np.sum(d**2, axis=0) for d in datas]
        norms = [ np.sqrt(np.sum(ssq)) for ssq in ssqs ]
        normed = [ data/norm for (data, norm) in zip(datas, norms) ]
        source, target = normed

        ##------ STEP 2: Calculate optimal rotation ------##
        U, s, Vh = np.linalg.svd(np.dot(target.T, source),
                                 full_matrices=False)
        T = np.dot(Vh.T, U.T)

        ##---------------- STEP 3: Scaling ---------------##
        ss = sum(s)
        scale = ss * norms[1] / norms[0]
        proj = scale * T

        return proj

    ##------------- STEP 4: Transformation -----------##
    proj = fit(source, target)
    return np.dot(source, proj)


def HyperAlign(data):

    ##----------- STEP 1: MAKE TEMPLATE -----------##
    # make preliminary template from subject 1 and adjust it
    template = np.copy(data[0])
    for x in range(1, len(data)):
        next = procrustes(data[x], template/(x+1))
        template += next
    template /= len(data)

    ##-------- STEP 2: NEW COMMON TEMPLATE --------##
    # align each subj to the template from STEP 1
    new_template = np.zeros(template.shape)
    for x in range(0, len(data)):
        next = procrustes(data[x], template)
        new_template += next
    new_template /= len(data)

    ##---- STEP 3: HYPER-ALIGN TO NEW TEMPLATE ----##
    # align each subj to the Final template from STEP 2
    aligned = [np.zeros(new_template.shape)] * len(data)
    aligned = []
    for x in range(0, len(data)):
        next = procrustes(data[x], new_template)
        aligned.append(next)

    # data > Template > aligned data
    return aligned


In [ ]:
# count max voxels per roi
# With the help of Thirza Dado

# Define the regions of interest (ROIs)
rois = ["V1", "V2", "V3", "V4", "LOC", "FFA", "PPA"]
c1 = 0
c2 = 0

# Iterate over each ROI
for roi in rois:
    # Initialize an array to store the number of voxels for each subject
    num_voxels = np.zeros(3, int)
    xs = []
    # Load data for each subject
    for subject in range(3):
        # Load the voxel data for the current subject and ROI
        x = np.load(f"/content/drive/MyDrive/GOD-NEW-3p/x_{subject+1}_{roi}.npy")
        print(f"Shape of the data {np.array(x).shape}")
        # Store the number of voxels (second dimension of x) for the current subject
        num_voxels[subject] = x.shape[1]
        xs.append(x)

    # Find the index of the subject with the maximum number of voxels
    max_voxels_index = np.argmax(num_voxels)
    # If the subject with the maximum number of voxels is not the first subject
    if max_voxels_index != 0:
        xs.insert(0, xs.pop(max_voxels_index))

    # Get the maximum number of voxels
    max_num_voxels = num_voxels[max_voxels_index]

    # Perform hyperalignment on the data
    x_aligned = HyperAlign(xs)
    print(np.array(x_aligned).shape)

    #If you want them meaned
    #mean_along_first_axis = np.mean(x_aligned, axis=0)
    #print("Mean along the first axis shape:", x_aligned.shape)
    np.save(f"/content/drive/MyDrive/GOD-NEW-3p/{roi}_hyperaligned.npy", x_aligned)
    c1 += np.max(num_voxels)
    c2 += xs[0].shape[1]



Shape of the data (1250, 2521)
Shape of the data (1250, 2329)
Shape of the data (1250, 1418)
(3, 1250, 2521)
Shape of the data (1250, 2370)
Shape of the data (1250, 2687)
Shape of the data (1250, 1498)
(3, 1250, 2687)
Shape of the data (1250, 1765)
Shape of the data (1250, 2058)
Shape of the data (1250, 1470)
(3, 1250, 2058)
Shape of the data (1250, 1283)
Shape of the data (1250, 837)
Shape of the data (1250, 1257)
(3, 1250, 1283)
Shape of the data (1250, 2257)
Shape of the data (1250, 1871)
Shape of the data (1250, 2724)
(3, 1250, 2724)
Shape of the data (1250, 2100)
Shape of the data (1250, 1084)
Shape of the data (1250, 662)
(3, 1250, 2100)
Shape of the data (1250, 767)
Shape of the data (1250, 900)
Shape of the data (1250, 614)
(3, 1250, 900)
